In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

KeyboardInterrupt: 

In [6]:
df = pd.read_csv('Datos-1.csv')
df.head(5)

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176


In [11]:
# “¿Cuánto ‘compra’ la felicidad el dinero?”
# Haz un scatter plot entre Economy (GDP per Capita) y Happiness Score con línea de tendencia. 
# Cuenta si el retorno marginal parece decreciente y destaca un outlier.

fig1 = px.scatter(df ,x='Economy (GDP per Capita)', y='Happiness Score', color='Region', trendline='ols')
fig1.show()


Analizando el grafico podemos decir que para la mayoria de paises el dinero influye en la felicidad de manera directa, sin embargo, en la región de Norte America hay una excepción dado que los datos muestran que a mayor Economy menor Happiness Score por lo que esta regón es un outlier con repsecto a las otras tendencias.

In [14]:
# “Top 10: ¿liderazgo estable o frágil?”
# Muestra el Top-10 países con mejor Happiness Rank en un gráfico de líneas ordenado. 
# Narra si hay “brechas de oro” entre los primeros tres y el resto

fig2 = px.line(df.head(10), x='Country', y='Happiness Score')
fig2.show()

El grafico muestra que el liderazgo del top 3 es estable, ya que sus puntajes de felicidad son practicamente iguales, alrededor de 7.5 y 7.6. Se observa una brecha de oro con respecto al resto, quizas no tan marcada pero si a una ligera distancia que los diferencia esta se ubica entre Noruega y Canadá. A partir del cuarto lugar los niveles siguen altos aunque con un liderazgo mas frágil en comparación con los primeros 3.

In [4]:
# “¿Qué región ‘sonríe’ más?”
# Construye un bar chart con el promedio de Happiness Score por Region. 
# Cuenta qué región lidera y cuál queda rezagada. Incluye barras de error (Standard Error).
r3 = df.groupby('Region')['Happiness Score'].mean().reset_index()

std_r3 = df.groupby('Region')['Happiness Score'].std().reset_index()
std_r3.rename(columns={'Happiness Score': 'std'}, inplace=True)

r3 = r3.merge(std_r3, on='Region')
fig3 = px.bar(r3, x='Region' ,y='Happiness Score', color_discrete_sequence=["#d4c600"], error_y='std')
fig3.show()

En el gráfico se ve que norte america es la región que lidera en Happiness Score, seguida de cerca por Australia y nueva zelanda y Western Europe.
La región más rezagada es Sub-Saharan Africa, con el promedio más bajo de felicidad.

In [ ]:
# “Anatomía de un país feliz”
# Para el país #1 en Happiness Rank, arma un stacked bar con los factores (Economy, Family, Health, Freedom, Trust, Generosity). Explica qué pilar pesa más.

factores = ["Economy (GDP per Capita)", "Family", "Health (Life Expectancy)", "Freedom", "Trust (Government Corruption)", "Generosity"]

top1 = df.sort_values("Happiness Rank", ascending=True).iloc[0]
country_name = top1["Country"]

vals = top1[factores].astype(float)
plot_df = pd.DataFrame({"Factor": factores, "Valor": vals.values})
plot_df["Country"] = country_name

fig4 = px.bar(
    plot_df,
    x="Country",
    y="Valor",
    color="Factor",
    orientation="v",
    barmode="stack",
    text=plot_df["Valor"].round(2)
)

fig4.update_layout(
    title=f"Anatomía de la felicidad – {country_name} (Top 1)",
    xaxis_title="Contribución al Happiness Score",
    yaxis_title="",
    legend_title="Pilar"
)

fig4.show()


El pilar que más pesa es Economy (GDP per Capita) con un aporte de 1.40.Le sigue muy de cerca Family, que aporta 1.35.
Esto muestra que la base económica sólida es el principal soporte del alto Happiness Score de Suiza.

RESPUESTA

In [ ]:
df_region = df.groupby("Region")[factores].mean().reset_index()

# Normalización por región
df_region_norm = df_region.copy()
df_region_norm[factores] = (df_region[factores] - df_region[factores].min()) / (df_region[factores].max() - df_region[factores].min())

# Heatmap por regiones
fig5 = px.imshow(
    df_region_norm[factores].values,
    labels=dict(x="Factor", y="Región", color="Valor normalizado"),
    x=factores,
    y=df_region_norm["Region"],
    color_continuous_scale="Plasma"
)

fig5.update_layout(title="Mapa de calor de similitudes por Región")
fig5.show()



In [ ]:
# Visualizacion 6
import numpy as np

factors = ["Economy (GDP per Capita)", "Family", "Health (Life Expectancy)", "Freedom", "Trust (Government Corruption)", "Generosity"]

# Filtrar columnas y quitar nulos
dfm = df[["Country", "Region"] + factors].dropna()

# Normalización (z-score)
X = dfm[factors].astype(float).to_numpy()
Xz = (X - X.mean(axis=0)) / X.std(axis=0, ddof=0)

countries = dfm["Country"].to_numpy()
regions   = dfm["Region"].to_numpy()

# Ordenar países (por 1ª componente principal)
U, S, VT = np.linalg.svd(Xz, full_matrices=False)
proj = (Xz @ VT.T)[:,0]
order = np.argsort(proj)

Xo, Co, Re = Xz[order], countries[order], regions[order]

# Buscar "hermanos inesperados"
D = np.linalg.norm(Xo[:,None,:] - Xo[None,:,:], axis=2)
np.fill_diagonal(D, np.inf)
mask = np.not_equal.outer(Re, Re)
if mask.any():
    Dm = np.where(mask, D, np.inf)
    i, j = np.unravel_index(np.argmin(Dm), Dm.shape)
else:
    i, j = np.unravel_index(np.argmin(D), D.shape)

print(f"Hermanos inesperados: {Co[i]} ({Re[i]}) {Co[j]} ({Re[j]}) distancia = {D[i,j]:.3f}")

# Heatmap con px.imshow
fig6 = px.imshow(
    Xo,
    x=factors,
    y=Co,
    color_continuous_scale="RdBu_r",
    labels={"color": "z-score"},
    title="Hermandades ocultas entre países – Heatmap clusterizado (z-score)",
    aspect="auto"   # <-- evita que quede tan delgado
)

# Resaltar hermanos inesperados
for r in [i, j]:
    fig6.add_shape(
        type="rect",
        x0=-0.5, x1=len(factors)-0.5,
        y0=r-0.5, y1=r+0.5,
        line=dict(color="orange", width=3)
    )

fig6.update_layout(   # más ancho
    height=1200    # más alto
)

fig6.show()

Hermanos inesperados: Canada (North America) Netherlands (Western Europe) distancia = 0.264


En el heatmap se resaltan con un recuadro naranja Australia y Netherlands como los dos países que aparecen como “hermanos inesperados"Aunque pertenecen a regiones distintas, su perfil en los factores de felicidad resulta muy similar, lo que los hace destacar como pareja atípica en el cluster.

In [ ]:
# Visualización 7

# Histograma de Happiness Score con box marginal
fig7 = px.histogram(
    df,
    x="Happiness Score",   # usa exactamente el nombre de tu Excel
    nbins=30,
    marginal="box",
    opacity=0.85,
    title="Distribución de Happiness Score"
)
fig7.update_layout(xaxis_title="Happiness Score", yaxis_title="Frecuencia")
fig7.show()

In [ ]:
# Visualización 8

# Boxplot de Happiness Score por Region
fig8 = px.box(
    df,
    x="Region",              
    y="Happiness Score",     
    points="outliers",       
    title="Heterogeneidad de Happiness Score por Región"
)

fig8.update_layout(
    xaxis_title="Región",
    yaxis_title="Happiness Score",
    xaxis_tickangle=45
)

fig8.show()


LATAM y caribbean tiene una mediana de Happiness Score ligeramente por encima de la global, lo que refleja un nivel medio-alto en comparación mundial, aunque un outlier con puntaje bajo evidencia la desigualdad interna en la región.

R

In [ ]:
# 9

# Scatter Freedom vs Trust con histogramas marginales y líneas de mediana
fig9 = px.scatter(
    df,
    x="Freedom",                 # usa el nombre exacto de tu Excel
    y="Trust (Government Corruption)",  # ajusta al nombre exacto en tu archivo
    color="Region",
    hover_name="Country",
    hover_data={"Happiness Score": True},
    marginal_x="histogram",
    marginal_y="histogram",
    title="Libertad vs. Confianza"
)

# Calcular medianas
x_med = df["Freedom"].median()
y_med = df["Trust (Government Corruption)"].median()

# Trazar líneas guía
fig9.add_vline(x=x_med, line_dash="dash", line_color="black")
fig9.add_hline(y=y_med, line_dash="dash", line_color="black")

fig9.update_layout(
    xaxis_title="Freedom",
    yaxis_title="Trust",
    legend_title="Region"
)

fig9.show()

In [ ]:
# Visualización 10


# Detectar etiqueta de LATAM (busca "latin" en Region, sin depender de un nombre fijo)
regions = df["Region"].astype(str)
latam_candidates = [r for r in regions.unique() if "latin" in r.lower()]
latam_label = latam_candidates[0] if latam_candidates else regions.unique()[0]

# Orden: LATAM primero
cat_order = [latam_label] + [r for r in regions.unique() if r != latam_label]

# Violin plot con box=True y LATAM primero
fig10 = px.violin(
    df,
    x="Region",                 # usa exactamente los nombres de tu Excel
    y="Happiness Score",
    box=True,
    category_orders={"Region": cat_order},
    points='all',
    title="Violines de felicidad por Región (LATAM destacada)"
)


y_med_latam = df.loc[regions == latam_label, "Happiness Score"].median()
fig10.add_annotation(
    x=latam_label, y=y_med_latam,
    text="LATAM",
    yshift=30
)

fig10.update_layout(xaxis_title="Región", yaxis_title="Happiness Score")
fig10.show()

Las regiones con mayor dispersión son Middle East and Northern Africa y Latin America and Caribbean, y la mediana ajusta el ranking al mostrar que en LATAM el nivel típico es más alto de lo que indica el promedio.

In [ ]:
# Pregunta 11 Visualización Nº2

# Promedio de Generosity por Región
r11 = df.groupby("Region", as_index=False)["Generosity"].mean()

# Ordenar descendente
r11 = r11.sort_values("Generosity", ascending=False)

# Gráfico de barras
fig11 = px.bar(
    r11,
    x="Region",
    y="Generosity",
    title="Promedio de Generosity por Región",
    color="Region",
    text=r11["Generosity"].round(3),
)

fig11.update_layout(
    xaxis_title="Región",
    yaxis_title="Promedio de Generosity",
    xaxis_tickangle=45,
    showlegend=False,
    font=dict(size=9)
)

fig11.show()

In [ ]:
# Pregunta 11 Visualización Nº2


# Promedio de Generosity por Región
r12 = df.groupby("Country", as_index=False)[["Generosity",'Happiness Score']].mean()

# Ordenar descendente
r12 = r12.sort_values("Generosity", ascending=False)

# Gráfico de barras
fig12 = px.bar(
    r12.head(10),
    x="Country",
    y="Generosity",
    title="Promedio de Generosity por Región",
    color="Country",
    text=r12.head(10)["Generosity"].round(3),
    hover_data=['Happiness Score']
)

fig12.update_layout(
    xaxis_title="Región",
    yaxis_title="Promedio de Generosity",
    xaxis_tickangle=45,
    showlegend=False,
    font=dict(size=9)
)

fig12.add_shape(
    type="rect",
    x0=-0.5, x1=1.5,   # de la primera a la segunda barra (índices 0 y 1 en el eje X)
    y0=0, y1=r12["Generosity"].max()*1.05,  # cubrir hasta un poco más del valor máximo
    line=dict(color="red", width=2),
    fillcolor="rgba(255,0,0,0.1)",  # relleno semitransparente
    layer="below"  # para que no tape las barras
)

fig12.show()

In [74]:
list(df.columns)

['Country',
 'Region',
 'Happiness Rank',
 'Happiness Score',
 'Standard Error',
 'Economy (GDP per Capita)',
 'Family',
 'Health (Life Expectancy)',
 'Freedom',
 'Trust (Government Corruption)',
 'Generosity',
 'Dystopia Residual']

In [75]:
# Pregunta 12

# Columnas EXACTAS según tu Excel
cols = [
    "Economy (GDP per Capita)",
    "Family",
    "Health (Life Expectancy)",
    "Freedom",
    "Trust (Government Corruption)",
    "Generosity",
    "Happiness Score"
]

# Matriz de correlación (Pearson)
corr = df[cols].corr(method="pearson")

# Heatmap de correlaciones
fig = px.imshow(
    corr,
    x=corr.columns,
    y=corr.index,
    color_continuous_scale="RdBu_r",
    zmin=-1, zmax=1,
    labels={"color": "Correlación (r)"},
    title="Matriz de correlación – Factores y Happiness Score"
)

fig.update_layout(
    xaxis_title="Variables",
    yaxis_title="Variables"
)

fig.show()

La relación más fuerte es entre Economy (GDP per Capita) y Happiness Score, mientras que la más débil aparece con Generosity.
Una posible hipótesis es que la prosperidad económica favorece condiciones de vida que incrementan la felicidad; en cambio, la generosidad refleja más un rasgo cultural que acompaña, pero no determina directamente el nivel de felicidad.

In [78]:
# Pregunta 13

vars_disc = [
    "Economy (GDP per Capita)",
    "Health (Life Expectancy)",
    "Freedom",
    "Trust (Government Corruption)"
]
score_col = "Happiness Score"

# Copia y elimina nulos
dfc = df[vars_disc + [score_col]].dropna().copy()

# Discretización en Bajo/Medio/Alto (terciles)
labels3 = ["Bajo", "Medio", "Alto"]
for c in vars_disc:
    dfc[c + "_q"] = pd.qcut(dfc[c], q=3, labels=labels3, duplicates="drop")

# Categoría de felicidad (bajo/alto) por mediana
med = dfc[score_col].median()
dfc["Happiness_q"] = pd.cut(
    dfc[score_col],
    bins=[-float("inf"), med, float("inf")],
    labels=["Bajo", "Alto"],
    include_lowest=True
)

# Parallel categories (rutas categorizadas)
fig = px.parallel_categories(
    dfc,
    dimensions=["Happiness_q"] + [c + "_q" for c in vars_disc],
    color=score_col,
    color_continuous_scale="deep",
    labels={
        "Happiness_q": "Happiness Score",
        "Economy (GDP per Capita)_q": "Economy",
        "Health (Life Expectancy)_q": "Health",
        "Freedom_q": "Freedom",
        "Trust (Government Corruption)_q": "Trust"
    },
    title="Rutas categorizadas (Bajo/Medio/Alto) – Economy, Health, Freedom, Trust vs Happiness Score"
)

fig.update_layout(coloraxis_colorbar=dict(title="Happiness Score"))
fig.show()

In [79]:
# Filtra filas completas en las columnas usadas

cols = ["Health (Life Expectancy)", "Family", "Happiness Score", "Region", "Country", "Economy (GDP per Capita)"]
df3d = df[cols].dropna().copy()

# Scatter 3D: Salud (x), Familia (y), Felicidad (z), color por Región
fig = px.scatter_3d(
    df3d,
    x="Health (Life Expectancy)",
    y="Family",
    z="Happiness Score",
    color="Region",
    hover_name="Country",
    hover_data={
        "Economy (GDP per Capita)": ':.3f',
        "Health (Life Expectancy)": ':.3f',
        "Family": ':.3f',
        "Happiness Score": ':.3f',
        "Region": False,  # ya está por color
        "Country": False  # ya está en hover_name
    },
    title="Triada crítica: Salud + Familia + Felicidad"
)

fig.update_layout(
    scene=dict(
        xaxis_title="Health (Life Expectancy)",
        yaxis_title="Family",
        zaxis_title="Happiness Score"
    ),
    legend_title="Region"
)

fig.show()

El gráfico muestra que al superar umbrales altos en Health y Family, los países tienden a ubicarse en la parte superior del eje de felicidad.
Se forma así una meseta de alta felicidad, con puntajes sostenidos por encima de 6.
Esto indica que salud y familia actúan como pilares básicos para mantener la felicidad elevada.

In [84]:

factores = [
    "Economy (GDP per Capita)",
    "Family",
    "Health (Life Expectancy)",
    "Freedom",
    "Trust (Government Corruption)",
    "Generosity"
]

# Detecta la etiqueta de LATAM exactamente como aparece en tu Excel
regions = df["Region"].astype(str)
latam_label = next((r for r in regions.unique() if "latin" in r.lower()), None)
assert latam_label is not None, "No se encontró una región que contenga 'latin' en su nombre."

# Promedios Global y LATAM
global_mean = df[factores].mean(numeric_only=True)
latam_mean  = df.loc[regions == latam_label, factores].mean(numeric_only=True)

# Formato largo para polar
radar_df = pd.DataFrame({
    "Factor": factores * 2,
    "Valor": pd.concat([latam_mean, global_mean], axis=0).values,
    "Grupo": ["LATAM"] * len(factores) + ["Global"] * len(factores)
})

# Radar (polar) con dos trazas
fig = px.line_polar(
    radar_df,
    r="Valor",
    theta="Factor",
    color="Grupo",
    line_close=True,
    category_orders={"Factor": factores},
    title="LATAM vs. Mundo – Promedios de factores"
)
fig.update_traces(mode="lines+markers")
fig.update_traces(fill="toself")  # relleno tipo radar
fig.update_layout(legend_title_text="")
fig.show()

En LATAM los pétalos de Family y Generosity se abren más que el promedio global, mientras que Economy, Health y Trust se cierran notablemente. Esto muestra un fuerte sostén social, pero con rezagos en capital económico, salud e instituciones.